<a href="https://colab.research.google.com/github/hrishikesav/BigData/blob/master/EmployeeAttendanceCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving dept-data.txt to dept-data.txt


In [5]:
!{'mkdir -p data'}
!ls

data  dept-data.txt  sample_data


In [7]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')

>> pip install --quiet apache-beam



**Sequential transformation**

In [21]:
import apache_beam as beam

p1 = beam.Pipeline()

lines = (p1
         | beam.io.ReadFromText('dept-data.txt')
         | beam.Map(lambda element: element.split(','))
         | beam.Filter(lambda element: element[3] == 'Accounts')
         | beam.Map(lambda element: (element[1],1))
         | beam.CombinePerKey(sum)
         | beam.io.WriteToText('data/sample_out')
         )
p1.run()

!{('head -n 20 data/sample_out-00000-of-00001')}


('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


**Parallel transformation**

In [29]:
import apache_beam as beam

p1 = beam.Pipeline()

master_collection = (p1
         | "Read input" >> beam.io.ReadFromText('dept-data.txt')
         | "Split based on comma" >> beam.Map(lambda element: element.split(','))
)

accounts_count = (master_collection
        | "Filter accounts" >> beam.Filter(lambda element: element[3] == 'Accounts')
        | "Add count 1 for accounts" >> beam.Map(lambda element: ("Accounts, " + element[1],1))
        | "Group and sum accounts" >> beam.CombinePerKey(sum)
        #| "Write accounts count to file" >> beam.io.WriteToText('data/accounts')
    
)

hr_count = (master_collection
        | "Filter HR" >> beam.Filter(lambda element: element[3] == 'HR')
        | "Add count 1 for HR" >> beam.Map(lambda element: ("HR, " + element[1],1))
        | "Group and sum HR" >> beam.CombinePerKey(sum)
        #| "Write HR to file" >> beam.io.WriteToText('data/hr')
    
)

combine_count = ((accounts_count,hr_count)
        | "Combine counts" >> beam.Flatten()
        | "Write to single file" >> beam.io.WriteToText('data/combine')
)
p1.run()

#!{('head -n 20 data/accounts-00000-of-00001')}
#!{('head -n 20 data/hr-00000-of-00001')}

!{('head -n 20 data/combine-00000-of-00001')}


('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


**ParDo function transform**

In [35]:
import apache_beam as beam

class SplitElement(beam.DoFn):
  def process(self,element):
    return [element.split(',')]

p1 = beam.Pipeline()

lines = (p1
         |"Read input" >> beam.io.ReadFromText('dept-data.txt')
         #| beam.ParDo(SplitElement())
         # or ParDo using lambda
         | "ParDo with lambda" >> beam.ParDo(lambda element: [element.split(',')])
         #wrong| "ParDo with lamba filter" >> beam.ParDo(lambda element: [element[3] == 'Accounts'])
         #| beam.Map(lambda element: (element[1],1))
         #| beam.CombinePerKey(sum)
         | "Write output" >> beam.io.WriteToText('data/sample_out')
         )
p1.run()

!{('head -n 20 data/sample_out-00000-of-00001')}


['149633CM', 'Marco', '10', 'Accounts', '1-01-2019']
['212539MU', 'Rebekah', '10', 'Accounts', '1-01-2019']
['231555ZZ', 'Itoe', '10', 'Accounts', '1-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '1-01-2019']
['704275DC', 'Kyle', '10', 'Accounts', '1-01-2019']
['957149WC', 'Kyle', '10', 'Accounts', '1-01-2019']
['241316NX', 'Kumiko', '10', 'Accounts', '1-01-2019']
['796656IE', 'Gaston', '10', 'Accounts', '1-01-2019']
['331593PS', 'Beryl', '20', 'HR', '1-01-2019']
['560447WH', 'Olga', '20', 'HR', '1-01-2019']
['222997TJ', 'Leslie', '20', 'HR', '1-01-2019']
['171752SY', 'Mindy', '20', 'HR', '1-01-2019']
['153636AS', 'Vicky', '20', 'HR', '1-01-2019']
['745411HT', 'Richard', '20', 'HR', '1-01-2019']
['298464HN', 'Kirk', '20', 'HR', '1-01-2019']
['783950BW', 'Kaori', '20', 'HR', '1-01-2019']
['892691AR', 'Beryl', '20', 'HR', '1-01-2019']
['245668UZ', 'Oscar', '20', 'HR', '1-01-2019']
['231206QD', 'Kumiko', '30', 'Finance', '1-01-2019']
['357919KT', 'Wendy', '30', 'Finance', '1-01-2019